In [2]:
  
from keras.models import load_model
import cv2
import numpy as np
from random import choice
from tensorflow import keras
import matplotlib.pyplot as plt


In [1]:
REV_CLASS_MAP = {
    3: "none",
    0: "paper",
    1: "rock",
    2: "scissors"
    
}


def mapper(val):
    return REV_CLASS_MAP[val]

In [3]:
#loading saved model
new_model = keras.models.load_model("my_model_5")


In [4]:
def calculate_winner(move1, move2):
    if move1 == move2:
        return "Tie"

    if move1 == "rock":
        if move2 == "scissors":
            return "User"
        if move2 == "paper":
            return "Computer"

    if move1 == "paper":
        if move2 == "rock":
            return "User"
        if move2 == "scissors":
            return "Computer"

    if move1 == "scissors":
        if move2 == "paper":
            return "User"
        if move2 == "rock":
            return "Computer"

In [13]:
cam = cv2.VideoCapture(0)
img_counter=0
prev_move="None"
while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
   
    k = cv2.waitKey(1)
    cv2.rectangle(frame, (40, 80), (260, 300), (225, 255, 225), 2)
    cv2.rectangle(frame, (400, 80), (620, 300), (255, 255, 255), 2)
    roi = frame[80:300, 40:260] #(y1:y2),(x1,x2) {region of interest}
    
    #processing image as per our model
    
    img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (112, 112))
    
    
    # predict the move made
    
    pred = new_model.predict(np.array([img]))
    move_code = np.argmax(pred[0])
    user_move_name = mapper(move_code)
    
    
     # predict the winner (user vs computer) :
    
    if prev_move != user_move_name:
        computer_move_name = choice(['rock', 'paper', 'scissors'])
        winner = calculate_winner(user_move_name, computer_move_name)
        prev_move = user_move_name

     
    #printing all results :
    
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, "Your Move: " + user_move_name,
                (50, 350), font, 0.65, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (370, 350), font, 0.65, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.putText(frame, "Winner: " + winner,
                (220, 420), font, 1, (0, 0, 255), 3, cv2.LINE_AA)
    
    
    
    icon = cv2.imread("icon/{}.png".format(computer_move_name))
    icon = cv2.resize(icon, (220, 220))
    frame[80:300, 400:620] = icon
        
     
    cv2.imshow("JoDD", frame)
    
    # press "q" for quiz:
    
    if k == ord('q'):
        break
    

cam.release()

cv2.destroyAllWindows()